# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [115]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter, defaultdict
import textdistance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [116]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [117]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

In [118]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [119]:
N = sum(vocab.values())
def P(word, N=N):
    return vocab[word] / N

In [120]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [121]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [122]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    return closest

In [113]:
def get_closest_hybrid_match_considering_probas(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    # группируем слова по расстоянию редактирования
    result = defaultdict(list)
    for word in closest:
        result[word[1]].append(word[0])
    
    # если у слов одинаковое расстояние, считаем вероятность для каждого и оставляем слово с максимальной вероятностью 
    for options in list(result.values()):
        if len(options) > 1:
            probas = []
            for option in options:
                probas.append((P(option), option))
            options.clear()
            options.append(max(probas)[1])
            
    # возвращаем список кортежей    
    return list(zip([i[0] for i in list(result.values())], list(result.keys())))

### Сравним результаты без учета с и учетом вероятности 

In [11]:
print(get_closest_hybrid_match('солнечный', X, vec))
print(get_closest_hybrid_match_considering_probas('солнечный', X, vec))

[('солнечный', 1.0), ('солнечные', 0.8888888888888888), ('солнечным', 0.8888888888888888)]
[('солнечный', 1.0), ('солнечные', 0.8888888888888888)]


In [12]:
print(get_closest_hybrid_match('колокол', X, vec))
print(get_closest_hybrid_match_considering_probas('колокол', X, vec))

[('колокол', 1.0), ('колокола', 0.875), ('колоколе', 0.875)]
[('колокол', 1.0), ('колокола', 0.875)]


In [13]:
print(get_closest_hybrid_match('компьютер', X, vec))
print(get_closest_hybrid_match_considering_probas('компьютер', X, vec))

[('компьютер', 1.0), ('компьютеры', 0.9), ('компьютеру', 0.9)]
[('компьютер', 1.0), ('компьютеру', 0.9)]


## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [28]:
def generate_deletes(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes

In [57]:
deletes_dict = defaultdict(list)
for k, v in vocab.items():
    dels = generate_deletes(k)
    for option in dels:
        deletes_dict[option].append(k)
# all_corrections - список со всеми возможными удалениями для всех слов из словаря     
deletes_list = list(deletes_dict.keys())

In [61]:
def correction(word):
    if word in vocab:
        return word
    elif word in deletes_list:
        return max(deletes_dict[word], key = P)
    else:
        dels = generate_deletes(word)
        if word in deletes_list:
            return max(deletes_dict[word], key = P)

In [83]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [84]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8133066533266633
0.15993788819875776
0.09004249454461927
